In [53]:
# imports and link xml
import re
from math import radians, cos, sin, asin, sqrt
import folium
from IPython.display import display
import pandas as pd
from pprint import pprint
import xml.etree.ElementTree as ET
tree = ET.parse('session/Kismet-20191011-18-31-55-1.netxml')
root = tree.getroot()

In [3]:
StationList = [] # MAC, [[min GPS],[max GPS]], [[first seen],[last seen]], device type, encryption, ESSID

In [4]:
def getDeviceType(bssid): # find vendor based off MAC address
    file = open('mac-vendors.txt', 'r', encoding="utf8")
    raw=[]
    mac_dict={}
    for line in file:
        raw.append(re.split(r'\t+', line.strip()))
    for pair in raw:
        mac_dict[pair[0]]= pair[1]
    return mac_dict.get(bssid[:8])

In [9]:
def haversine(lon1, lat1, lon2, lat2): # haversine formula
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = 6371* c/ 1.069
    return m

In [5]:
def appendArray(net, child): # parse xml and create array data
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        if (net=="wireless-network"): # network xml tag
            scan= 'BSSID'
        else:
            scan= 'client-mac'        # client xml tag
        for bssid_tag in child.findall(scan):
            bssid = bssid_tag.text
        for gps_tag in child.findall("gps-info"):
            gps= [[gps_tag[0].text, gps_tag[1].text],[gps_tag[4].text, gps_tag[5].text]]
        time = [child.attrib['first-time'],child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'):
                essid= essid_tag.text
            for info_tag in ssid_tag :
                if (info_tag.tag == "encryption"):
                    encryption.append(info_tag.text)
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid]) 

In [6]:
for child in root:
    appendArray("wireless-network",child) #networks
    for subchild in child:
        appendArray("wireless-client", subchild) #clients

In [7]:
pd.DataFrame(StationList, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

,BSSID,GPS,Dates,Manufacturer,Encryption,ESSID
0,00:02:6F:22:86:21,"[[34.195740, -118.340050], [34.195740, -118.34...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:36:36...","Senao International Co., Ltd.",[None],Downtown-Burbank
1,00:02:6F:22:86:21,"[[34.195740, -118.340050], [34.195740, -118.34...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:36:36...","Senao International Co., Ltd.",[],
2,00:0D:97:00:59:05,"[[34.191360, -118.341759], [34.191360, -118.34...","[Fri Oct 11 18:38:10 2019, Fri Oct 11 18:38:10...",ABB Inc./Tropos,[],
3,00:0D:97:00:59:05,"[[34.191360, -118.341759], [34.191360, -118.34...","[Fri Oct 11 18:38:10 2019, Fri Oct 11 18:38:10...",ABB Inc./Tropos,[],
4,00:0D:97:00:5A:92,"[[34.190983, -118.341637], [34.191032, -118.34...","[Fri Oct 11 18:40:30 2019, Fri Oct 11 18:41:52...",ABB Inc./Tropos,[None],BWP Free WiFi
...,...,...,...,...,...,...
1216,FC:EC:DA:E9:DB:61,"[[34.195740, -118.340050], [34.195740, -118.34...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:36:36...",Ubiquiti Networks Inc.,[],
1217,FE:65:DE:9C:27:90,"[[34.196274, -118.340141], [34.196274, -118.34...","[Fri Oct 11 18:35:36 2019, Fri Oct 11 18:35:36...",None,"[WPA+PSK, WPA+AES-CCM]",None
1218,FE:65:DE:9C:27:90,"[[34.196274, -118.340141], [34.196274, -118.34...","[Fri Oct 11 18:35:36 2019, Fri Oct 11 18:35:36...",None,[],
1219,FE:EC:DA:E9:DB:61,"[[34.196003, -118.340233], [34.196003, -118.34...","[Fri Oct 11 18:33:33 2019, Fri Oct 11 18:33:33...",None,"[WPA+PSK, WPA+AES-CCM]",Commonwealth


In [46]:
for index in range(0,len(StationList)):
    tmp = haversine(float(StationList[index][1][0][0]),float(StationList[index][1][0][1]),float(StationList[index][1][1][0]),float(StationList[index][1][1][1]))
    if (tmp>.25):
        print(StationList[index])


['00:0D:97:00:5B:33', [['34.190987', '-118.341560'], ['34.196323', '-118.340012']], ['Fri Oct 11 18:35:27 2019', 'Fri Oct 11 18:45:34 2019'], 'ABB Inc./Tropos', ['None'], 'BWP Free WiFi']
['00:0D:97:00:5B:33', [['34.190987', '-118.341560'], ['34.196323', '-118.340012']], ['Fri Oct 11 18:35:27 2019', 'Fri Oct 11 18:45:34 2019'], 'ABB Inc./Tropos', [], '']
['00:0D:97:09:B1:A4', [['34.190887', '-118.341789'], ['34.195740', '-118.340050']], ['Fri Oct 11 18:36:36 2019', 'Fri Oct 11 18:45:43 2019'], 'ABB Inc./Tropos', ['None'], None]
['00:0D:97:09:B1:A4', [['34.190887', '-118.341789'], ['34.195740', '-118.340050']], ['Fri Oct 11 18:36:36 2019', 'Fri Oct 11 18:45:43 2019'], 'ABB Inc./Tropos', ['None'], '']
['8C:45:00:0C:F2:95', [['34.190895', '-118.341751'], ['34.196323', '-118.340096']], ['Fri Oct 11 18:32:48 2019', 'Fri Oct 11 18:45:39 2019'], None, [], '']
['8C:45:00:0C:F2:95', [['34.190895', '-118.341751'], ['34.196323', '-118.340096']], ['Fri Oct 11 18:32:48 2019', 'Fri Oct 11 18:45:39 2

In [57]:
LDN_COORDINATES = (34.196136, -118.340164)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=60) 
display(myMap)